# Encoding Categorical Variables

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## Category Encoders

A set of scikit-learn-style transformers for encoding categorical variables into numeric with different techniques.

https://contrib.scikit-learn.org/category_encoders/

Install: `category_encoders`

## Creating binary variables through one-hot encoding

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data = pd.read_csv('data/creditApprovalUCI.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['A16'], axis=1), data['A16'], test_size=0.3, random_state=0)

In [ ]:
X_train['A4'].unique()

In [ ]:
encoder = OneHotEncoder(categories='auto', drop='first', sparse=False)

In [ ]:
vars_categorical = ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']
encoder.fit(X_train[vars_categorical])

In [ ]:
X_train_enc = encoder.transform(X_train[vars_categorical])
X_test_enc = encoder.transform(X_test[vars_categorical])

In [ ]:
X_train_enc

## Performing one-hot encoding of frequent categories

In [ ]:
from feature_engine.encoding import OneHotEncoder
from feature_engine.imputation import CategoricalImputer

In [ ]:
data = pd.read_csv('data/creditApprovalUCI.csv')

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['A16'], axis=1), # predictors
    data['A16'], # target
    test_size=0.3, # percentage of observations in test set
    random_state=0) # seed to ensure reproducibility


imputer = CategoricalImputer()
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [ ]:
X_train['A6'].unique()

In [ ]:
X_train['A6'].value_counts().sort_values(ascending=False).head(5)

In [ ]:
ohe_enc = OneHotEncoder(top_categories=5, variables=['A6', 'A7'], drop_last=False)

In [ ]:
ohe_enc.fit(X_train)

In [ ]:
X_train_enc = ohe_enc.transform(X_train)
X_test_enc = ohe_enc.transform(X_test)

In [ ]:
X_train_enc.head()

In [ ]:
ohe_enc.encoder_dict_

## Replacing categories with ordinal numbers

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
data = pd.read_csv('data/creditApprovalUCI.csv')

X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['A16'], axis=1), data['A16'],test_size=0.3, random_state=0)

In [ ]:
vars_categorical = ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

le = OrdinalEncoder()

In [ ]:
le.fit(X_train[vars_categorical])

In [ ]:
X_train_enc = le.transform(X_train[vars_categorical])
X_test_enc = le.transform(X_test[vars_categorical])

## Replacing categories with counts or frequency of observations

In [ ]:
data = pd.read_csv('data/creditApprovalUCI.csv')

In [ ]:
data.head(3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['A16'], axis=1), data['A16'],test_size=0.3, random_state=0)

imputer = CategoricalImputer()
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [ ]:
from feature_engine.encoding import CountFrequencyEncoder

count_enc = CountFrequencyEncoder(encoding_method='count', variables=None)

In [ ]:
count_enc.fit(X_train)

In [ ]:
count_enc.encoder_dict_

In [ ]:
X_train_enc = count_enc.transform(X_train)
X_test_enc = count_enc.transform(X_test)

In [ ]:
X_train_enc.head()

## Encoding with integers in an ordered manner

In [ ]:
data = pd.read_csv('data/creditApprovalUCI.csv')

X_train, X_test, y_train, y_test = train_test_split(data, data['A16'], test_size=0.3, random_state=0)

imputer = CategoricalImputer()
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [ ]:
X_train["A7"].unique()

In [ ]:
X_train.groupby(['A7'])['A16'].mean().plot()
plt.title('Relationship between A7 and the target')
plt.ylabel('Mean of target')
plt.show()

In [ ]:
X_train.groupby(['A7'])['A16'].mean()

In [ ]:
X_train.groupby(['A7'])['A16'].mean().sort_values()

In [ ]:
ordered_labels = X_train.groupby(['A7'])['A16'].mean().sort_values().index
ordered_labels

In [ ]:
ordinal_mapping = {k: i for i, k in enumerate(ordered_labels, 0)}
ordinal_mapping

In [ ]:
X_train['A7'] = X_train['A7'].map(ordinal_mapping)
X_test['A7'] = X_test['A7'].map(ordinal_mapping)

In [ ]:
X_train.groupby(['A7'])['A16'].mean().plot()
plt.title('Relationship between A7 and the target')
plt.ylabel('Mean of target')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['A16'], axis=1), data['A16'], test_size=0.3, random_state=0)

In [ ]:
from feature_engine.encoding import OrdinalEncoder

ordinal_enc = OrdinalEncoder(encoding_method='ordered', variables=None)

In [ ]:
ordinal_enc.fit(X_train, y_train)

In [ ]:
ordinal_enc.encoder_dict_

In [ ]:
X_train_enc = ordinal_enc.transform(X_train)
X_test_enc = ordinal_enc.transform(X_test)

In [ ]:
X_test_enc.head()

## Encoding with the mean of the target

In [ ]:
data = pd.read_csv('data/creditApprovalUCI.csv')

X_train, X_test, y_train, y_test = train_test_split(data, data['A16'], test_size=0.3, random_state=0)

imputer = CategoricalImputer()
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [ ]:
from feature_engine.encoding import MeanEncoder

mean_enc = MeanEncoder(variables=None)

In [ ]:
mean_enc.fit(X_train, y_train)

In [ ]:
X_train_enc = mean_enc.transform(X_train)
X_test_enc = mean_enc.transform(X_test)

## Grouping rare or infrequent categories

In [ ]:
from feature_engine.encoding import RareLabelEncoder

In [ ]:
data = pd.read_csv('data/creditApprovalUCI.csv')

X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['A16'], axis=1), data['A16'],test_size=0.3, random_state=0)

imputer = CategoricalImputer()
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [ ]:
rare_encoder = RareLabelEncoder(tol=0.05, n_categories=4)

In [ ]:
rare_encoder.fit(X_train)

In [ ]:
X_train_enc = rare_encoder.transform(X_train)
X_test_enc = rare_encoder.transform(X_test)